In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor

C:\Users\pf910\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [121]:
wday = pd.read_csv("Scaled_feature_pickup.csv")

In [122]:
nopickup = pd.read_csv("PhaseI_Labels_all.csv")
wday = wday.merge(nopickup, on= "BoroCT2010", how = "inner")

###  RF Predition 

In [5]:
regressor = RandomForestRegressor()
gr = GridSearchCV(regressor, param_grid={'max_depth': range(1,22)})

In [10]:
p = []
f = []
rs = []
q = []
n = []
nodata = []
wday1 = pd.DataFrame()
satisfied = pd.DataFrame()
demanded = pd.DataFrame()
nopickup_x = pd.DataFrame()
satisfied_x = pd.DataFrame()
satisfied_y = pd.DataFrame()
demanded_x = pd.DataFrame()


for i in range(13, 24):
    r= 0
    rf_gr = 0
    wday1 = wday[(wday.iloc[:,i+16]!='no_pickup_value')]
    wday1 = wday.reset_index(drop = True)
    r  = np.percentile(wday.iloc[:,i].fillna(0).values, 96)
    wday1 = wday1[wday1.iloc[:,i]<r]#Removing outliers
    wday1.iloc[:,i] = wday1.iloc[:,i]/2
    p.append(wday1.iloc[:,i].sum())
    satisfied = wday1[wday1.iloc[:,i+16]=='Not_Considered']
    satisfied = satisfied.reset_index(drop = True)
    f.append(satisfied.iloc[:,i+16].sum())
    demanded = wday1[wday1.iloc[:,i+16]!='Not_Considered']
    demanded =demanded.reset_index(drop = True)
    nopickup_x = nopickup[[u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    satisfied_x = satisfied[[ u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    satisfied_y = satisfied.iloc[:,i]
    dx= demanded[['BoroCT2010']]
    demanded_x = demanded[[u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    q.append(demanded.iloc[:,i].sum())
    rf_gr = gr.fit(satisfied_x, satisfied_y)
    rs.append(rf_gr.best_params_)
    forest = 0
    forest = regressor.fit(satisfied_x, satisfied_y)
    pred_y1 = 0
    pred_y1 = regressor.predict(demanded_x)
    demanded_x['pred'+str(i)] = pred_y1
    demanded['tripdemanded'+str(i)] = demanded_x['pred'+str(i)] - demanded.iloc[:,i]
    demanded = demanded[demanded['tripdemanded'+str(i)]>0]
    n.append(demanded['tripdemanded'+str(i)].sum())
    forest = 0
    forest = regressor.fit(satisfied_x, satisfied_y)
    pred_y2 = 0
    pred_y2 = regressor.predict(nopickup_x)
    nopickup['predpickup'+str(i)] = pred_y2
    nodata.append(nopickup['predpickup'+str(i)].sum())

C:\Users\pf910\Anaconda2\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\pf910\Anaconda2\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Function to find the prediction for each time slot

In [14]:
def carto(i) :
    
    wday1 = wday[(wday.iloc[:,i+16]!='no_pickup_value')]
    wday1 = wday.reset_index(drop = True)
    r  = np.percentile(wday.iloc[:,i].fillna(0).values, 96)
    wday1 = wday1[wday1.iloc[:,i]<r]
    wday1.iloc[:,i] = wday1.iloc[:,i]/2
    p.append(wday1.iloc[:,i].sum())
    satisfied = wday1[wday1.iloc[:,i+16]=='Not_Considered']
    satisfied = satisfied.reset_index(drop = True)
    demanded = wday1[wday1.iloc[:,i+16]!='Not_Considered']
    demanded =demanded.reset_index(drop = True)
    nopickup_x = nopickup[[u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    satisfied_x = satisfied[[ u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    satisfied_y = satisfied.iloc[:,i]
    demanded_ = demanded[['BoroCT2010']]
    demanded_x = demanded[[u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    rf_gr = gr.fit(satisfied_x, satisfied_y)
    forest = 0
    forest = regressor.fit(satisfied_x, satisfied_y)
    pred_y1 = 0
    pred_y1 = regressor.predict(demanded_x)
    demanded_x['pred'+str(i)] = pred_y1
    demanded['tripdemanded'+str(i)] = demanded_x['pred'+str(i)] - demanded.iloc[:,i]
    demanded = demanded[demanded['tripdemanded'+str(i)]>0]
    demanded_['tripdemanded'] = demanded['tripdemanded'+str(i)]
    forest = 0
    forest = regressor.fit(satisfied_x, satisfied_y)
    pred_y2 = 0
    pred_y2 = regressor.predict(nopickup_x)
    nopickup_ = nopickup[['BoroCT2010']]
    nopickup_['predpickup'+str(i)] = pred_y2
    pred_y2

In [119]:
nopickup_xin = nopickup[['BoroCT2010', u'predpickup13', u'predpickup14', u'predpickup15', u'predpickup16',u'predpickup17', u'predpickup18', u'predpickup19', u'predpickup20',u'predpickup21', u'predpickup22', u'predpickup23']]

In [30]:
wday1 = pd.DataFrame()
satisfied = pd.DataFrame()
demanded = pd.DataFrame()
nopickup_x = pd.DataFrame()
satisfied_x = pd.DataFrame()
satisfied_y = pd.DataFrame()
demanded_x = pd.DataFrame()

In [31]:
dx = pd.DataFrame(columns = ["BoroCT2010", "tripdemanded"])

In [32]:

def carto_trip(i):
    r= 0
    wday1 = wday[(wday.iloc[:,i+16]!='no_pickup_value')]
    wday1 = wday.reset_index(drop = True)
    r  = np.percentile(wday.iloc[:,i].fillna(0).values, 96)
    wday1 = wday1[wday1.iloc[:,i]<r]
    wday1.iloc[:,i] = wday1.iloc[:,i]/2
    satisfied = wday1[wday1.iloc[:,i+16]=='Not_Considered']
    satisfied = satisfied.reset_index(drop = True)
    demanded = wday1[wday1.iloc[:,i+16]!='Not_Considered']
    demanded =demanded.reset_index(drop = True)
    satisfied_x = satisfied[[ u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    satisfied_y = satisfied.iloc[:,i]
    demanded_x = demanded[[u'Street_betweenness_centrality_avg', u'Subway_counts_daily', u'income',
       u'OfficeArea', u'ResArea', u'ComArea', u'RetailArea', u'safety',
       u'pop_density', u'employee', u'rent']]
    forest = 0
    forest = regressor.fit(satisfied_x, satisfied_y)
    pred_y1 = 0
    pred_y1 = regressor.predict(demanded_x)
    demanded_x['pred'+str(i)] = pred_y1
    demanded['tripdemanded'+str(i)] = demanded_x['pred'+str(i)] - demanded.iloc[:,i]
    demanded = demanded[demanded['tripdemanded'+str(i)]>0]
    dx= demanded[['BoroCT2010']]
    dx['tripdemanded'+str(i)] = demanded['tripdemanded'+str(i)]
    return(dx)

### Finding the trip demanded for BoroCT 4122701

In [187]:
l=[]
for i in range (13, 24):
    dx1 = carto_trip(i)
    l.append(dx1[dx1['BoroCT2010']==4122701]['tripdemanded'+str(i)].values)

C:\Users\pf910\Anaconda2\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\pf910\Anaconda2\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [188]:
l 

[array([ 356.9]),
 array([ 898.9]),
 array([ 1023.05]),
 array([ 396.15]),
 array([ 408.]),
 array([ 1691.8]),
 array([ 670.55]),
 array([ 24.3]),
 array([ 381.3]),
 array([ 72.3]),
 array([ 230.4])]

In [189]:
#wday.iloc[['BoroCT2010']==4099704][i+16].values
#wday.loc[wday['BoroCT2010']==4099704 ,i+16]
wday2 = wday
wday2 = wday2.set_index(['BoroCT2010'])

In [191]:
c = []
for i in range(12,23):
    c.append(wday2.ix[4122701][i])

In [192]:
c

[10.0, 7.0, 15.0, 6.0, 5.0, 12.0, 10.0, 3.0, 9.0, 0.0, 2.0]

In [193]:
ct_final = pd.DataFrame(columns = ['Time slot','current number of trip', 'trip demanded'])

In [194]:
ct_final['Time slot'] = e
ct_final['current number of trip'] = c
ct_final['trip demanded'] = l

In [195]:
ct_final['trip demanded'] = ct_final['trip demanded'].astype(str)

In [196]:
ct_final['trip demanded'] = (ct_final['trip demanded'].str[1:-1])

In [198]:
ct_final.to_csv("bc4122701.csv")

### Creating dataset for total number of predictions

In [20]:
finaltable = pd.DataFrame(columns = {'time slot','current number of trips',
                                     'trip demanded in under served areas','current number of trip in underserved areas'
                                     , 'prediction of trips in no data areas', 'percentage of demanded trip'})
e = wday.columns.tolist()[13:24]
len(e)

In [22]:
finaltable['time slot'] = e

In [23]:
finaltable['trip demanded in under served areas'] = n
finaltable['prediction of trips in no data areas'] = nodata
finaltable['current number of trips'] = p
finaltable['current number of trip in underserved areas'] = q
finaltable['percentage of demanded trip'] = (finaltable['trip demanded in under served areas']/
                                             finaltable['current number of trips'])*100

In [28]:
finaltable.head(2)

,current number of trip in underserved areas,current number of trips,prediction of trips in no data areas,trip demanded in under served areas,time slot,percentage of demanded trip
0,1546.5,293213.0,1055.3,23324.7,wd00_06,7.954866
1,1175.0,420617.5,703.9,23534.8,wd06_10,5.595297


In [29]:
finaltable.to_csv("final.csv")